In [ ]:
import random
import numpy as np
import torch

In [ ]:
from main_ae import AgeModel
from main_ae import START_AGE, END_AGE, NUM_AGE_GROUPS

In [ ]:
NUM_AGE_GROUPS = 9
VALIDATION_RATE = 0.1

In [ ]:
random.seed(2019)
np.random.seed(2019)
torch.manual_seed(2019)

# Model

In [ ]:
num_ages = END_AGE - START_AGE + 1
model = AgeModel(num_ages, NUM_AGE_GROUPS)  # age_pred, age_group_pred

# Dataloader

In [ ]:
from data import NiaDataset
import numpy as np
from PIL import Image
import os

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision

In [ ]:
train_meta_path = "nia_cropped/train_0.npy"
test_meta_path = "nia_cropped/test_0.npy"

In [ ]:
batch_size = 16

## NiaDataset

### Train dataloader

In [ ]:
transforms_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomApply(
            [
                torchvision.transforms.RandomAffine(degrees=10, shear=16),
                torchvision.transforms.RandomHorizontalFlip(p=1.0),
            ],
            p=0.5,
        ),
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.RandomCrop((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

In [ ]:
train_gen = NiaDataset(train_meta_path, transforms_train)

In [ ]:
train_loader = DataLoader(
    dataset=train_gen,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=0,
)

In [ ]:
train_iter = iter(train_loader)

In [ ]:
sample = next(train_iter)

In [ ]:
sample.keys()

In [ ]:
sample['age_class']

### Validation dataloader

In [ ]:
transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)
val_gen = NiaDataset(test_meta_path, transforms)
val_loader = DataLoader(
    val_gen, batch_size=1, shuffle=False, pin_memory=True, num_workers=0
)

# Optimizer

In [ ]:
from torch import optim
import torch
from torch.optim import lr_scheduler

In [ ]:
LAMBDA_1 = 0.2
LAMBDA_2 = 0.05
START_AGE = 0
END_AGE = 90
learning_rate = 1e-3
epoch = 2

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5, 8, 9], gamma=0.1)

# Loss function

In [ ]:
from mean_variance_loss import MeanVarianceLoss

In [ ]:
criterion1 = MeanVarianceLoss(LAMBDA_1, LAMBDA_2, START_AGE, END_AGE).cuda
criterion2 = torch.nn.CrossEntropyLoss().cuda()

# Train

In [ ]:
from main_ae import train_softmax, evaluate_softmax

In [ ]:
def train_softmax(train_loader, model, criterion2, optimizer, epoch, result_directory):
    model.train()
    running_loss = 0.
    running_softmax_loss = 0.
    interval = 1
    for i, sample in enumerate(train_loader):
        images = sample['image'].cuda()
        labels = sample['age_class'].cuda()
        _, output = model(images)
        loss = criterion2(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.data
        if (i + 1) % interval == 0:
            print('[%d, %5d] loss: %.3f'
                  % (epoch, i, running_loss / interval))
            with open(os.path.join(result_directory, 'log'), 'a') as f:
                f.write('[%d, %5d] loss: %.3f\n'
                        % (epoch, i, running_loss / interval))
            running_loss = 0.

In [ ]:
for epoch in range(2):
    train_softmax(train_loader, model, criterion2, optimizer, epoch, "result")
    loss_val, mae = evaluate_softmax(val_loader, model, criterion2)
    scheduler.step()

In [ ]:
loss_val, mae